In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (1e-04, 0.1)
epsilon_range = (1e-05, 0.001)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    return creator.Individual([C, epsilon])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 140
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 40
crossover_rate = 0.1
mutation_rate = 0.9

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [0.03887903776509694, 0.00062120303016896], MAPE: 56.679308403766
Individu: [0.0001, 0.001], MAPE: 56.46532712741631
Individu: [0.06868628117891278, 0.001], MAPE: 56.17926406984931
Individu: [0.09132475596099537, 0.0009418234467260596], MAPE: 57.6593085410997
Individu: [0.1, 0.0005024482964628594], MAPE: 58.41187066868766
Individu: [0.014974473552719716, 0.0005526770297860187], MAPE: 56.616394798787574
Individu: [0.1, 0.00042541282672059656], MAPE: 58.418156530196406
Individu: [0.08484944530833893, 0.0008388019477623926], MAPE: 57.06698526138524
Individu: [0.06807098021855999, 0.0004876950622764792], MAPE: 56.15840638104104
Individu: [0.0001, 0.0007655521655291016], MAPE: 56.453301372521295
Individu: [0.02964908649341665, 0.0009168855499127603], MAPE: 56.812812088073386
Individu: [0.07385851804644362, 0.0004741339561225474], MAPE: 56.35459437806761
Individu: [0.1, 0.0005703813235038284], MAPE: 58.40632753899327
Individu: [0.0001, 0.0006561778747143473], MAPE: 56.4

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[10.53353342]
 [ 9.27133784]
 [ 9.10304509]
 [ 9.08060605]
 [ 9.07761418]
 [ 9.07721526]]
